In [1]:
import torchvision
import torch
import sys

sys.path.append("..")

from torch.utils.data import DataLoader, Dataset
from utils.model_util import LeNet5, CNN, MLP
from utils.train_util import *

In [2]:
model = LeNet5(28, 28, 1, 10).cuda()
train_dataset_loader = DataLoader(
    dataset=torchvision.datasets.MNIST(
        root='../data/raw-data/',
        train=True,
        transform=torchvision.transforms.ToTensor()),
    batch_size=600,
    shuffle=False)
test_dataset_loader = DataLoader(
    dataset=torchvision.datasets.mnist.MNIST(
        root='../data/raw-data/',
        transform=torchvision.transforms.ToTensor()),
    batch_size=32,
    shuffle=False)
test_dataset = torchvision.datasets.mnist.MNIST(
    root='../data/raw-data/',
    transform=torchvision.transforms.ToTensor())

opti = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
for data, target in train_dataset_loader:
    opti.zero_grad()
    output = model(data.cuda())
    loss = criterion(output, target.cuda())
    loss.backward()
    print(loss.item())
    opti.step()

print(loss, type(loss))
print(loss.item(), type(loss.item()))

1387.6546630859375
1385.5897216796875
1381.2158203125
1382.1663818359375
1382.4404296875
1378.5506591796875
1376.6419677734375
1374.6002197265625
1372.0262451171875
1367.2415771484375
1362.730712890625
1361.0374755859375
1358.40234375
1346.23291015625
1337.410888671875
1326.302734375
1308.9014892578125
1302.1546630859375
1284.792236328125
1271.6314697265625
1257.94873046875
1232.860107421875
1201.1700439453125
1163.3885498046875
1144.0504150390625
1091.7138671875
1064.5404052734375
1018.1316528320312
988.1824340820312
969.216064453125
881.040283203125
846.0673828125
754.8175048828125
756.2213134765625
715.6175537109375
677.116943359375
616.1062622070312
653.5068359375
544.3605346679688
568.1482543945312
552.0859375
565.171630859375
448.8465881347656
530.3010864257812
560.0924682617188
531.82080078125
424.9000244140625
464.9433898925781
454.2672424316406
486.6104431152344
446.0724182128906
427.84521484375
512.3550415039062
449.49951171875
405.20953369140625
433.8404541015625
295.3622436

In [3]:
model = LeNet5(28, 28, 1, 10).cuda()
train_dataset_loader = DataLoader(
    dataset=torchvision.datasets.MNIST(
        root='../data/raw-data/',
        train=True,
        transform=torchvision.transforms.ToTensor()),
    batch_size=600,
    shuffle=False)
test_dataset_loader = DataLoader(
    dataset=torchvision.datasets.mnist.MNIST(
        root='../data/raw-data/',
        transform=torchvision.transforms.ToTensor()),
    batch_size=32,
    shuffle=False)

opti = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
loss_sum = 0
for data, target in train_dataset_loader:
    opti.zero_grad()
    output = model(data.cuda())
    loss = criterion(output, target.cuda())
    # print(f'loss_sum = {loss_sum:<20}', end=' | ')
    loss.backward()
    loss_sum += loss.item()
    # print(f'loss = {loss.item():<20}')
    opti.step()

In [4]:
def eval_model(model, dataset, device):
    '''
    评估模型
    '''
    model_copy = deepcopy(model).eval()
    correct = 0
    data_loader = DataLoader(dataset, batch_size=32)
    for images, targets in data_loader:
        outputs = model_copy(images.to(device))
        _, predicted = torch.max(outputs, 1)
        correct += torch.eq(predicted, targets.to(device)).sum()
    # print(f'Test Accuracy: {100 * correct / total:.2f}%')
    accuracy = correct / len(dataset)
    return accuracy

eval_model(model, test_dataset, 'cuda')

tensor(0.8725, device='cuda:0')

In [5]:
model_copy = deepcopy(model).eval()
model_copy.eval()
model_copy.cuda()
correct = 0
total = 0
data_loader = test_dataset_loader
for images, targets in data_loader:
    outputs = model_copy(images.cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets.cuda()).sum().item()
    break
print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

Test Accuracy: 84.38%


In [6]:
for i in range(8, 13):
    print(f'{i:<10}|')
    print(f'{i:.2f}|')

8         |
8.00|
9         |
9.00|
10        |
10.00|
11        |
11.00|
12        |
12.00|


In [3]:
def get_list(len_list, term=[]):
    '''
    返回一个全是空列表的列表
    '''
    list_return = []
    for _ in range(len_list):
        list_return.append(term)
    return list_return

num_all_client = 6
num_all_server = 2

server_model = get_list(num_all_server)
server_accuracy = get_list(num_all_server)
client_accuracy = get_list(num_all_client)
server_client_model = get_list(num_all_server)
server_model_distillation_accuracy = get_list(num_all_server)

print(client_accuracy)

[[], [], [], [], [], []]
